In [1]:
# gui_app/main.py
import sys
import traceback
from PyQt6.QtWidgets import QApplication
from PyQt6.QtCore import QThread, pyqtSignal
from main_window import MainWindow
import cpp_backend

class BackendThread(QThread):
    """后台线程，运行C++计算"""
    result_ready = pyqtSignal(object)
    error_occurred = pyqtSignal(str)
    
    def __init__(self):
        super().__init__()
        self.calculator = cpp_backend.Calculator()
        self.image_processor = cpp_backend.ImageProcessor()
        self.task = None
        self.args = ()
        self.kwargs = {}
    
    def set_task(self, func_name, *args, **kwargs):
        self.task = getattr(self, func_name, None)
        self.args = args
        self.kwargs = kwargs
    
    def run(self):
        try:
            if self.task:
                result = self.task(*self.args, **self.kwargs)
                self.result_ready.emit(result)
        except Exception as e:
            self.error_occurred.emit(str(e))
    
    def calculate_expression(self, expr):
        return self.calculator.calculate_expression(expr)
    
    def process_image(self, image_path, operation, **params):
        img = self.image_processor.load_image(image_path)
        if operation == "resize":
            return self.image_processor.resize_image(img, 
                params.get('width', 800), 
                params.get('height', 600))
        elif operation == "grayscale":
            return self.image_processor.convert_to_grayscale(img)
        elif operation == "filter":
            return self.image_processor.apply_filter(img, 
                params.get('filter_type', 'gaussian'))
        return img

class MainApp:
    def __init__(self):
        self.app = QApplication(sys.argv)
        self.backend_thread = BackendThread()
        self.main_window = MainWindow()
        
        # 连接信号
        self.backend_thread.result_ready.connect(self.on_result_ready)
        self.backend_thread.error_occurred.connect(self.on_error)
        
        # 设置窗口的后端引用
        self.main_window.set_backend(self.backend_thread)
    
    def run(self):
        self.main_window.show()
        return self.app.exec()

if __name__ == "__main__":
    app = MainApp()
    sys.exit(app.run())


AttributeError: module 'cpp_backend' has no attribute 'Calculator'